<a href="https://colab.research.google.com/github/krissme28/data_analysis/blob/main/%D0%9F%D1%80%D0%BE%D0%B5%D0%BA%D1%821.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import psycopg2
import pandas as pd
import numpy as np
import sqlalchemy as sa
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
%matplotlib inline

In [ ]:
conn = psycopg2.connect(
    host="158.160.52.106",
    port=5432,
    database="postgres",
    user="student",
    password="JvLda93aA",
)
cur = conn.cursor()

In [ ]:
df = pd.read_sql("""select to_char(visit_dttm, 'YYYY-MM') as month,
                                  count(distinct client_rk) as visit,
                                  count(distinct ac.account_rk) as reg,
                                  count(ap.application_rk) as app,
                                  count(g.game_flg) filter(where g.game_flg = 1) as fact
                                FROM msu_analytics.client c
                                  left join msu_analytics.account ac using(client_rk)
                                  left join msu_analytics.application ap using(account_rk)
                                  left join msu_analytics.game g using(game_rk)
                                   GROUP BY month
                                   ORDER BY month DESC""", conn)
df = df.assign(konv1 = (df.reg /df.visit)*100)
df = df.assign(konv2 = (df.app /df.visit)*100)
df = df.assign(konv3 = (df.fact /df.visit)*100)
df

/usr/local/lib/python3.9/dist-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,month,visit,reg,app,fact,konv1,konv2,konv3
0,2023-02,32,8,0,0,25.000000,0.000000,0.000000
1,2023-01,852,213,111,21,25.000000,13.028169,2.464789
2,2022-12,805,196,100,39,24.347826,12.422360,4.844720
3,2022-11,786,200,101,71,25.445293,12.849873,9.033079
4,2022-10,843,217,109,74,25.741400,12.930012,8.778173
5,2022-09,778,190,91,59,24.421594,11.696658,7.583548


In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x = df.month, y = df.konv1, name='Из посещения сайта в регистрацию'))
fig.add_trace(go.Bar(x = df.month, y = df.konv2, name='Из посещения сайта в заявку'))
fig.add_trace(go.Bar(x = df.month, y = df.konv3, name='Из посещения сайта в прохождение игры'))
fig.update_layout(
    title="Воронка от посещения сайта по месяцам",
    title_x = 0.5,
    xaxis_title="step",
    yaxis_title="percent",
    legend=dict(x=.5, xanchor="center", orientation="h"),
    margin=dict(l=0, r=0, t=30, b=0))

fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.month, y=df.visit))
fig.update_layout(
    title="Количество посетителей",
    title_x = 0.5,
    xaxis_title="месяц",
    yaxis_title="количество",
    legend=dict(x=.5, xanchor="center", orientation="h"),
    margin=dict(l=0, r=0, t=30, b=0))
fig.show()